In [1]:
import sys
import os
import logging
import json
from dotenv import load_dotenv

# Change cwd to the DataEnrichment root folder
data_enrichment_root_path = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
os.chdir(data_enrichment_root_path)

if data_enrichment_root_path not in sys.path:
    sys.path.append(data_enrichment_root_path)

load_dotenv()

import db
from worker import process
from utils import download
from utils.get_video_description import get_video_description
from AudioProcessor.transcriber import transcribe_audio
from AudioProcessor.sentiment import SentimentAnalyzer
import AlignmentCalculator
import VideoProcessor
import AudioProcessor

/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db_conn = db.get_connection()
video_id = "7525069106249354518"

INFO - [EnrichmentWorker] - Successfully Connected to Database


In [3]:
video_path = os.path.abspath("./tmp/video/7525069106249354518.mp4")
audio_path = os.path.abspath("./tmp/video/7525069106249354518.wav")

transcript = """
Vă salut, dragi prieteni, vă salut, dragi români!
 Un filmuleț de aici, din Parlamentul, din Strasburg, din Parlamentul European,
 despre ce s-a întâmplat astăzi în sala de plen, în această clădire rotundă pe care o vedeți, asta de lemn, este mai rotundă.
 Dragi prieteni, astăzi, în plen, a fost un scandal.
 Un scandal pentru că partidele, Revue, Verzii și ceilalți, PSD, PNL, USR, mai departe,
 au invitat în plen, au invitat pe primarul Budapestei, exact pe primarul ăla, LGBTQ bananier,
 care a sfidat legea și ordonanța pe care domnul...
 prim-ministrul al Ungariei, Orban, a interzis parada LGBT.
 Primarul a lăsat și a acceptat această paradă de bananieri să vină în Budapesta.
 Un lucru foarte greu.
 În sală, aici, au început, cum am spus, grupul Revue, Verzii, EPP și ăștia le-au început să aplaude pe unguri, pe primarul Budapestei, așa, pe primarul Budapestei,
 și în momentul ăla, pentru că a ieșit oribil, oribil, într-o Uniune Europeană să susții LGBTQ, așa cum fac ei, este un lucru foarte grav.
 Și pentru asta, doamna Diana Ioanoviț-Oșoacă a dat tonul, a început să strige, să huiduie, să... nu să înjure, dar să strige contra primarului, alături de ea s-au alăturat și alți suveraniști, alți europaralători, alți...
 parlamentari, care susțin familia tradițională și familia compusă din bărbat și femeie, și vă dați seama că a ieșit un mic show în Parlamentul European, provocat de globaliști,
 provocat de acei extrimiști care ne fac pe noi, sau pe Pipera, pe Oșoacă, pe Luis Lazarus, pe naționaliști, îi fac extrimiști.
 Nu, extrimiștii sunt exact...
 cum își fondă line și toată armata ei, care își bat joc, pur și simplu își bat joc de cetățenii Uniunii Europene și de copiii Uniunii Europeni, cât și de pensionarii Uniunii Europene.
 Deci, este un lucru grav ce s-a întâmplat astăzi, dar ce să-i facem decât să ne spunem părerea și decât să îi huiduim înapoi?
 Mare lucru nu putem.
 Pentru că suntem noi în minoritate.
 Asta au fost aici din Parlamentul European de la Strasburg.
 Vă salut!
"""

sentiment_results = {
    'positive': 0.07932209223508835,
    'neutral': 0.0,
    'negative': 0.9206779599189758
 }

description = """
primarul din Budapesta huiduit de suveranisti in plenul de la Strasbourg #Romania #luislazarus #dianasosoaca #suveranitate #bananierii
"""

In [4]:
analysis_result_json = VideoProcessor.process(video_path, transcript, sentiment_results, description)
identified_subjects = analysis_result_json["identified_subjects"]
llm_overall_alignment = analysis_result_json["overall_alignment"]
analysis_result_json

INFO - [EnrichmentWorker] - Starting video processing for /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7525069106249354518.mp4
INFO - [EnrichmentWorker] - AFC is enabled with max remote calls: 10.
INFO - [EnrichmentWorker] - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/zeruel-ney/locations/global/publishers/google/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO - [EnrichmentWorker] - AFC remote call 1 is done.
INFO - [EnrichmentWorker] - VIDEO Analysis result {'summary': "The speaker shares a video from the European Parliament in Strasbourg, describing an incident where the Mayor of Budapest was allegedly booed by nationalist and sovereignist MEPs, including Diana Sosoaca. The speaker frames this event as a scandal, alleging that the Mayor defied the Hungarian Prime Minister's ban on an LGBTQ+ parade and that MEPs from Renew, Greens, EPP, and S&D groups applauded the Mayor. The speaker criticizes this support for 

{'summary': "The speaker shares a video from the European Parliament in Strasbourg, describing an incident where the Mayor of Budapest was allegedly booed by nationalist and sovereignist MEPs, including Diana Sosoaca. The speaker frames this event as a scandal, alleging that the Mayor defied the Hungarian Prime Minister's ban on an LGBTQ+ parade and that MEPs from Renew, Greens, EPP, and S&D groups applauded the Mayor. The speaker criticizes this support for LGBTQ+ rights as a sign of 'Western decay' and claims that 'globalists' and 'extremists' are mischaracterizing nationalists and patriots.",
 'identified_subjects': [{'subject': 'diana sosoaca', 'stance': 0.8},
  {'subject': 'viktor orbán', 'stance': 0.7},
  {'subject': 'european parliament', 'stance': -0.8},
  {'subject': 'european union', 'stance': -0.9},
  {'subject': 'greens', 'stance': -0.8},
  {'subject': 'renew europe', 'stance': -0.8},
  {'subject': "european people's party", 'stance': -0.8},
  {'subject': 'progressive allia

In [5]:
final_alignment, deterministic_alignment, alignment_conflict = AlignmentCalculator.calculate(identified_subjects, llm_overall_alignment, alpha=0.25)
print(f"llm_overall_alignment: {llm_overall_alignment} \nfinal_alignment: {final_alignment} \ndeterministic_alignment: {deterministic_alignment} \nalignment_conflict: {alignment_conflict}")

llm_overall_alignment: -0.98 
final_alignment: -0.9501011560693642 
deterministic_alignment: -0.8604046242774567 
alignment_conflict: 0.1195953757225433
